In [64]:
!gdown --id 1x5by4Snjh-qcch6UWFM0_6AvoryWVsT4
!unzip -qq /content/AIFORTHAI-ThaiWikiQA_NSC2020.zip # Q A
#output ThaiQACorpus-DevelopmentDataset.json, /content/wiki-documents-nsc.zip
!unzip -qq /content/wiki-documents-nsc.zip
#output /content/wiki-documents # D

Downloading...
From: https://drive.google.com/uc?id=1x5by4Snjh-qcch6UWFM0_6AvoryWVsT4
To: /content/AIFORTHAI-ThaiWikiQA_NSC2020.zip
100% 171M/171M [00:01<00:00, 102MB/s] 
replace ThaiQACorpus-DevelopmentDataset.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
replace wiki-documents-nsc/1.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


# Step 1: Load QAs

In [65]:
import os
import json
import pandas as pd

In [66]:
with open('ThaiQACorpus-DevelopmentDataset.json') as file:
    qa_dataset = json.load(file)
qa_dataset.keys()

dict_keys(['version', 'data_set_name', 'data'])

In [67]:
print(len(qa_dataset['data']))
qa_dataset['data'][:5] #115035

17000


[{'answer': 'ฮิกกิ้นส์',
  'answer_begin_position': 529,
  'answer_end_position': 538,
  'article_id': 115035,
  'question': 'สุนัขตัวแรกรับบทเป็นเบนจี้ในภาพยนตร์เรื่อง Benji ที่ออกฉายในปี พ.ศ. 2517 มีชื่อว่าอะไร',
  'question_id': 1,
  'question_type': 1},
 {'answer': 'เมชตา',
  'answer_begin_position': 139,
  'answer_end_position': 144,
  'article_id': 376583,
  'question': 'ลูนา 1 เป็นยานอวกาศลำแรกในโครงการลูนาของโซเวียต มีชื่อเรียกอีกชื่อว่าอะไร',
  'question_id': 2,
  'question_type': 1},
 {'answer': 'ลมสุริยะ',
  'answer_begin_position': 641,
  'answer_end_position': 649,
  'article_id': 376583,
  'question': 'ลูนา 1 เป็นยานอวกาศลำแรกในโครงการลูนาของโซเวียต สามารถสังเกตและตรวจวัดอะไรได้โดยตรง',
  'question_id': 3,
  'question_type': 1},
 {'answer': 'ปี 1933',
  'answer_begin_position': 304,
  'answer_end_position': 311,
  'article_id': 547560,
  'question': 'กระทรวงโฆษณาแถลงข่าวและโฆษณาชวนเชื่อของนาซีเยอรมนี ก่อตั้งขึ้นในปี ค.ศ. ใด',
  'question_id': 4,
  'question_type': 1},
 {'

In [68]:
qas = pd.DataFrame(qa_dataset['data'])
qas.head()

,question_id,question_type,question,answer,answer_begin_position,answer_end_position,article_id
0,1,1,สุนัขตัวแรกรับบทเป็นเบนจี้ในภาพยนตร์เรื่อง Ben...,ฮิกกิ้นส์,529.0,538.0,115035
1,2,1,ลูนา 1 เป็นยานอวกาศลำแรกในโครงการลูนาของโซเวีย...,เมชตา,139.0,144.0,376583
2,3,1,ลูนา 1 เป็นยานอวกาศลำแรกในโครงการลูนาของโซเวีย...,ลมสุริยะ,641.0,649.0,376583
3,4,1,กระทรวงโฆษณาแถลงข่าวและโฆษณาชวนเชื่อของนาซีเยอ...,ปี 1933,304.0,311.0,547560
4,5,1,ห้องทำงานรูปไข่ สร้างขึ้นในสมัยอดีตประธานาธิบด...,ประธานาธิบดีวิลเลียม เอช. ทัฟต์,231.0,262.0,193032


In [8]:
qas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   question_id            17000 non-null  int64  
 1   question_type          17000 non-null  int64  
 2   question               17000 non-null  object 
 3   answer                 17000 non-null  object 
 4   answer_begin_position  15000 non-null  float64
 5   answer_end_position    15000 non-null  float64
 6   article_id             17000 non-null  int64  
dtypes: float64(2), int64(3), object(2)
memory usage: 929.8+ KB


In [9]:
#make a unique list
used_article_ids = list(set([row.article_id for row in qas.itertuples()]))
len(used_article_ids)

10094

# Step 2: Load wiki documents

In [10]:
!pip install tqdm==4.60.0

     |████████████████████████████████| 75 kB 2.7 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.62.3
    Uninstalling tqdm-4.62.3:
      Successfully uninstalled tqdm-4.62.3


In [11]:
from tqdm.notebook import tqdm

## Version 1: Serial Programming

In [12]:
# #create a dataframe named wiki_documents
# wiki_documents = pd.DataFrame(sorted(os.listdir('wiki-documents')), columns=['file_name'])
# wiki_documents['article_id'] = wiki_documents['file_name'].map(lambda x: x.replace('.txt', '')).astype(int)
# wiki_documents['path'] = wiki_documents['file_name'].map(lambda x: os.path.join('wiki-documents', x))
# print(f'total rows: {wiki_documents.shape[0]}')
# wiki_documents.tail()

In [13]:
# #create a context column by reading from documents
# for doc in tqdm(wiki_documents.itertuples(index=True), total=wiki_documents.shape[0]):
# # for doc in wiki_documents.itertuples(index=True):
#     f = open(doc.path, mode='r', encoding='utf-8')
#     try:
#         wiki_documents.loc[doc.Index, 'context'] = f.read()
    
#     #some articles are unable to read due to decoding error
#     except UnicodeDecodeError as e:
#         print(str(e), doc.Index)
#         wiki_documents.loc[doc.Index, 'context'] = None
#     f.close()


# wiki_documents = wiki_documents.loc[~wiki_documents['context'].isna()] # i
# wiki_documents.reset_index(inplace=True, drop=True)
# print(f'total rows: {wiki_documents.shape[0]}')
# wiki_documents.tail()

## Version 2: Parallel Programming

In [14]:
len(os.listdir('/content/wiki-documents-nsc'))

125302

In [15]:
files = [f for f in os.listdir('/content/wiki-documents-nsc') if int(f.replace('.txt', '')) in used_article_ids]
len(files), files[:2]

(10094, ['11781.txt', '530931.txt'])

In [16]:
from multiprocessing.pool import ThreadPool

def read_file(input):
    file_name = input
    article_id = input.replace('.txt', '')
    path = '/content/wiki-documents-nsc/' + input

    with open(path, mode='r', encoding='utf-8') as file:
        try:
            content = file.read()
        except UnicodeDecodeError as e:
            content = ''
    
    return [file_name, article_id, path, content] 


In [17]:
results = ThreadPool(1000).imap_unordered(read_file, files)

In [18]:
wiki_documents = pd.DataFrame(results, columns=['file_name', 'article_id', 'path', 'content'])

In [19]:
wiki_documents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10094 entries, 0 to 10093
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   file_name   10094 non-null  object
 1   article_id  10094 non-null  object
 2   path        10094 non-null  object
 3   content     10094 non-null  object
dtypes: object(4)
memory usage: 315.6+ KB


In [20]:
#since the document content contain HTML syntax, we have to remove it
from io import StringIO
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

#example function usage
text = wiki_documents.loc[0, 'content']
print(text)
strip_tags(text)

<doc id="685790" url="https://th.wikipedia.org/wiki?curid=685790" title="คำวิเศษ">คำวิเศษ "คำวิเศษ" เป็นซิงเกิลของวงดนตรีเครสเชนโด้ ประพันธ์เนื้อร้องและแต่งทำนองโดย อุทัย บุนทรีรัตน์ และเรียบเรียงโดย นรเทพ มาแสง (นอ), ชินพัฒน์ หงส์อัมพร (แชมป์) เพลงนี้ยังรวมในอัลบั้ม Continue เนื้อหาของเพลงพูดถึงคำบางคำจากคนทั่ว ๆ ไป ได้ยินแล้วบางครั้งก็ลอยผ่านหายลับไปในอาก­าศ แต่คำบางคำจากคนบางคนถึงแม้จะเป็นเพียงคำส­ั้น ๆ คำธรรมดาแต่ทว่ามันช่างมีความหมายมากมายเห­ลือเกิน ทำให้โลกนี้น่าอยู่ ทำให้ท้องฟ้ากลับมาสดใส ทำให้พลังใจกลับมา ทำให้พลังกายกลับฟื้นคืน คำเหล่านั้นจากคนพิเศษมันคือคำที่สุดแสนวิ­เศษผู้มีส่วนร่วมทำเพลงผู้มีส่วนร่วมทำเพลง. - เครสเชนโด้ - นรเทพ มาแสง (นอ) – เบสกีตาร์ - ธีรพงษ์ ธนานิกกุล (จั๋ง) – เพอร์คัสชั่น - ชินพัฒน์ หงส์อัมพร (แชมป์) – กีตาร์ - ชาติชาย มานิตยกุล (นัท) – ร้องนำ ร้องประสาน - สุทธิพจน์ หวังวัฒนานุกุล (เลี้ยง) – กลองมิวสิกวีดิโอ มิวสิกวีดิโอ. มิวสิกวีดิโอเพลง "คำวิเศษ" ในเวอร์ชันแรกเผยแพร่ครั้งแรกในวันที่ 29 พฤษภาคม พ.ศ. 2555 กำกับโดย ทีม Eyedropper Fill เป็นเรื่องราวของความ

'คำวิเศษ "คำวิเศษ" เป็นซิงเกิลของวงดนตรีเครสเชนโด้ ประพันธ์เนื้อร้องและแต่งทำนองโดย อุทัย บุนทรีรัตน์ และเรียบเรียงโดย นรเทพ มาแสง (นอ), ชินพัฒน์ หงส์อัมพร (แชมป์) เพลงนี้ยังรวมในอัลบั้ม Continue เนื้อหาของเพลงพูดถึงคำบางคำจากคนทั่ว ๆ ไป ได้ยินแล้วบางครั้งก็ลอยผ่านหายลับไปในอาก\xadาศ แต่คำบางคำจากคนบางคนถึงแม้จะเป็นเพียงคำส\xadั้น ๆ คำธรรมดาแต่ทว่ามันช่างมีความหมายมากมายเห\xadลือเกิน ทำให้โลกนี้น่าอยู่ ทำให้ท้องฟ้ากลับมาสดใส ทำให้พลังใจกลับมา ทำให้พลังกายกลับฟื้นคืน คำเหล่านั้นจากคนพิเศษมันคือคำที่สุดแสนวิ\xadเศษผู้มีส่วนร่วมทำเพลงผู้มีส่วนร่วมทำเพลง. - เครสเชนโด้ - นรเทพ มาแสง (นอ) – เบสกีตาร์ - ธีรพงษ์ ธนานิกกุล (จั๋ง) – เพอร์คัสชั่น - ชินพัฒน์ หงส์อัมพร (แชมป์) – กีตาร์ - ชาติชาย มานิตยกุล (นัท) – ร้องนำ ร้องประสาน - สุทธิพจน์ หวังวัฒนานุกุล (เลี้ยง) – กลองมิวสิกวีดิโอ มิวสิกวีดิโอ. มิวสิกวีดิโอเพลง "คำวิเศษ" ในเวอร์ชันแรกเผยแพร่ครั้งแรกในวันที่ 29 พฤษภาคม พ.ศ. 2555 กำกับโดย ทีม Eyedropper Fill เป็นเรื่องราวของความรักความห่วงใย ระหว่าง แม่และลูกสาว ความเป็นตัวตนของลูกสาวและช่องว่างร

In [21]:
wiki_documents['content'] = wiki_documents['content'].map(lambda x: strip_tags(x))
wiki_documents.head()

,file_name,article_id,path,content
0,685790.txt,685790,/content/wiki-documents-nsc/685790.txt,"คำวิเศษ ""คำวิเศษ"" เป็นซิงเกิลของวงดนตรีเครสเชน..."
1,530931.txt,530931,/content/wiki-documents-nsc/530931.txt,ริโอบัมบา ริโอบัมบา () หรือชื่อเต็ม ซานเปโดรเด...
2,11781.txt,11781,/content/wiki-documents-nsc/11781.txt,วินสตัน เชอร์ชิล เซอร์วินสตัน ลีโอนาร์ด สเปนเซ...
3,627156.txt,627156,/content/wiki-documents-nsc/627156.txt,วุ้นมะพร้าว วุ้นมะพร้าว หรือ วุ้นสวรรค์ () เป็...
4,65293.txt,65293,/content/wiki-documents-nsc/65293.txt,ฝ้าย ฝ้าย เป็นพันธุ์ไม้สกุลกอสไซเพียมในวงศ์ชบา...


In [22]:
wiki_documents.to_csv('wiki_documents.csv', index=False)

In [23]:
# !cp 'wiki_documents.csv' {SAVE_DIR}

---

# Step 3: Fine-tuning QA model

In [24]:
qas.question_type.value_counts()

1    15000
2     2000
Name: question_type, dtype: int64

In [25]:
qas_t1 = qas.loc[qas.question_type == 1]
qas_t2 = qas.loc[qas.question_type == 2]
qas_t1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15000 entries, 0 to 14999
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   question_id            15000 non-null  int64  
 1   question_type          15000 non-null  int64  
 2   question               15000 non-null  object 
 3   answer                 15000 non-null  object 
 4   answer_begin_position  15000 non-null  float64
 5   answer_end_position    15000 non-null  float64
 6   article_id             15000 non-null  int64  
dtypes: float64(2), int64(3), object(2)
memory usage: 937.5+ KB


In [26]:
qas_t1.to_csv('qas_t1.csv', index=False)
# !cp 'qas_t1.csv' {SAVE_DIR}

qas_t2.to_csv('qas_t2.csv', index=False)
# !cp 'qas_t2.csv' {SAVE_DIR}

In [27]:
used_article_ids_t1 = list(set([str(row.article_id) for row in qas_t1.itertuples()]))
len(used_article_ids_t1)

8700

## 3.1 Build SQuAD

In [28]:
temp = []
for doc in wiki_documents.itertuples(index=True):
    if doc.article_id in used_article_ids_t1:
        temp.append(doc)
wiki_documents_t1 = pd.DataFrame(temp)
wiki_documents_t1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8700 entries, 0 to 8699
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Index       8700 non-null   int64 
 1   file_name   8700 non-null   object
 2   article_id  8700 non-null   object
 3   path        8700 non-null   object
 4   content     8700 non-null   object
dtypes: int64(1), object(4)
memory usage: 340.0+ KB


In [29]:
import numpy as np
np.random.seed(0) # for the same result

In [30]:
# #option 1: small dataset(n=1000) for demonstration
wiki_documents_t1_small = wiki_documents_t1.sample(n=1000, random_state=1234)

# #test train eval split
r = np.random.RandomState(1234)
split = r.rand(wiki_documents_t1_small.shape[0])
wiki_documents_t1_small.loc[split < 0.7, 'use_for'] = 'train'
wiki_documents_t1_small.loc[(0.7 < split) & (split < 0.8), 'use_for'] = 'eval'
wiki_documents_t1_small.loc[0.8 < split, 'use_for'] = 'test'
wiki_documents_t1_small['use_for'].value_counts()

train    683
test     219
eval      98
Name: use_for, dtype: int64

In [31]:
#option 2: total dataset: actual work
# test train eval split
# r = np.random.RandomState(1234)
# split = r.rand(wiki_documents_t1.shape[0])
# wiki_documents_t1.loc[split < 0.7, 'use_for'] = 'train'
# wiki_documents_t1.loc[(0.7 < split) & (split < 0.8), 'use_for'] = 'eval'
# wiki_documents_t1.loc[0.8 < split, 'use_for'] = 'test'
# wiki_documents_t1['use_for'].value_counts()

In [32]:
#make a SQUAD2.0 format dataset (visit: https://simpletransformers.ai/docs/qa-data-formats/)
question_id = 1
unknow_where_to_start = 0
dataset, train, eval, test, already = [], [], [], [], []
#assume one doc has just one article

# option 1: wiki_documents_t1_small, option 2: wiki_documents_t1
for article in tqdm(wiki_documents_t1_small.itertuples(), total=wiki_documents_t1_small.shape[0]): 
    document = {}
    document['context'] = article.content
    document['qas'] = []

    for qa in qas_t1.itertuples():
        if (article.article_id == str(qa.article_id)) and (qa.article_id not in already):
            already.append(qa.article_id)
            question = {}
            question['id'] = str(question_id)
            question_id += 1
            question['question'] = qa.question
            answer = {} #one question has just one answer
            answer['text'] = qa.answer 
            
            try:
                answer['answer_start'] = document['context'].index(answer['text'])
                question['is_impossible'] = False
            
            except:
                answer['answer_start'] = -1
                question['is_impossible'] = True
                unknow_where_to_start += 1
            
            question['answers'] = []
            question['answers'].append(answer)
            document['qas'].append(question)
            
            #may cause overhead while training, means same article, different question -> read twice
            if article.use_for == 'train':
                train.append(document)
            elif article.use_for == 'eval':
                eval.append(document)
            elif article.use_for == 'test':
                test.append(document)
    dataset.append(document)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [33]:
len(train), len(eval), len(test)

(683, 98, 219)

In [34]:
def reset_id(dataset):
    id = 1
    for doc in dataset:
        for qa in doc['qas']:
            qa['id'] = str(id)
            id += 1
            # print(id)
    return dataset

train, eval, test = reset_id(train), reset_id(eval), reset_id(test)

In [35]:
with open('train.json', 'w', encoding='utf-8') as file:
  json.dump(train, file, ensure_ascii=False)
with open('eval.json', 'w', encoding='utf-8') as file:
  json.dump(eval, file, ensure_ascii=False)
with open('test.json', 'w', encoding='utf-8') as file:
  json.dump(test, file, ensure_ascii=False)

In [36]:
# !cp 'train.json' {SAVE_DIR}
# !cp 'eval.json' {SAVE_DIR}
# !cp 'test.json' {SAVE_DIR}

## 3.2 Fine-tuning Model

In [37]:
with open('train.json', 'r') as file:
  train = json.load(file, encoding='utf-8')
with open('eval.json', 'r') as file:
  eval = json.load(file, encoding='utf-8')
with open('test.json', 'r') as file:
  test = json.load(file, encoding='utf-8')

In [38]:
!pip install -qq transformers
!pip install -qq simpletransformers

     |████████████████████████████████| 3.4 MB 5.2 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 596 kB 55.9 MB/s 
     |████████████████████████████████| 895 kB 50.4 MB/s 
     |████████████████████████████████| 3.3 MB 37.2 MB/s 
     |████████████████████████████████| 248 kB 5.1 MB/s 
     |████████████████████████████████| 311 kB 43.8 MB/s 
     |████████████████████████████████| 9.3 MB 50.8 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 1.7 MB 48.3 MB/s 
     |████████████████████████████████| 1.2 MB 50.9 MB/s 
     |████████████████████████████████| 180 kB 61.3 MB/s 
     |████████████████████████████████| 143 kB 52.3 MB/s 
     |████████████████████████████████| 97 kB 6.9 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 243 kB 55.8 MB/s 
     |████████████████████████████████| 133 kB 50.1 MB/s 
     |██████████████████

In [39]:
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.WARNING)

In [40]:
import torch
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

torch.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [41]:
# visit https://simpletransformers.ai/docs/usage/

model_args = {
    'manual_seed' : 0,
    'reprocess_input_data': False,
    
    'train_batch_size': 32,
    'eval_batch_size' : 8,
    
    'use_early_stopping': True,
    'early_stopping_delta': 0.01,
    'early_stopping_metric': 'eval_loss',
    'early_stopping_metric_minimize': True,
    'early_stopping_patience': 5,
    
    'evaluate_during_training' : True,
    'evaluate_during_training_verbose': True,
    
    'fp16': True,
    
    'num_train_epochs': 20,
    
    'overwrite_output_dir': True,

    'save_model_every_epoch': False,
    'save_steps':-1,
    
    'use_cached_eval_features' : True, 
    
    'max_seq_length': 256,
    'no_cache': False
}

In [42]:
#we can select a pretrained --> https://huggingface.co/models
#examples:
# 'monsoon-nlp/bert-base-thai'
# 'Geotrend/bert-base-th-cased'

In [43]:
bert = QuestionAnsweringModel(
    'bert', 
    'monsoon-nlp/bert-base-thai',
    use_cuda=torch.cuda.is_available(), 
    args=model_args
)

Downloading:   0%|          | 0.00/405 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/404M [00:00<?, ?B/s]

Some weights of the model checkpoint at monsoon-nlp/bert-base-thai were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from th

Downloading:   0%|          | 0.00/478k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/113 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [44]:
step, train_result = bert.train_model(train, eval_data=eval, show_running_loss=True)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
add example index and unique id: 100%|██████████| 683/683 [00:00<00:00, 536764.03it/s]


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Running Epoch 0 of 20:   0%|          | 0/29 [00:00<?, ?it/s]

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.

convert squad examples to features: 100%|██████████| 98/98 [00:02<00:00, 40.75it/s]

add example index and unique id: 100%|██████████| 98/98 [00:00<00:00, 261144.72it/s]


Running Evaluation:   0%|          | 0/16 [00:00<?, ?it/s]

...

Running Evaluation:   0%|          | 0/16 [00:00<?, ?it/s]

Running Epoch 19 of 20:   0%|          | 0/29 [00:00<?, ?it/s]

INFO:simpletransformers.question_answering.question_answering_model: Features loaded from cache at cache_dir/cached_dev_bert_256_98


Running Evaluation:   0%|          | 0/16 [00:00<?, ?it/s]

INFO:simpletransformers.question_answering.question_answering_model: Training of bert model complete. Saved to outputs/.


In [45]:
eval_result, eval_texts = bert.eval_model(eval)

INFO:simpletransformers.question_answering.question_answering_model: Features loaded from cache at cache_dir/cached_dev_bert_256_98


Running Evaluation:   0%|          | 0/16 [00:00<?, ?it/s]

In [46]:
eval_result

{'correct': 3, 'eval_loss': -6.166015625, 'incorrect': 48, 'similar': 47}

In [47]:
eval_texts['correct_text']

{'13': '19', '52': 'วุฒิชาติ ชุ่มสนิท', '77': 'ท่อฟรอนโทนาซัล'}

In [48]:
eval_texts['similar_text']

{'1': {'predicted': '',
  'question': 'การแข่งขันกีฬาวอลเลย์บอลชายชิงแชมป์โลก รุ่นอายุไม่เกิน 21 ปี ในปี ค.ศ. 1999 ถูกจัดขึ้นที่จังหวัดใดของประเทศไทย',
  'truth': 'อุบลราชธานี'},
 '15': {'predicted': 'อนันต์ กาญจนพาสน์ หรือ เสี่ยช้าง',
  'question': 'เจ้าของโครงการเมืองทองธานีมีชื่อว่าอะไร',
  'truth': 'อนันต์ กาญจนพาสน์'},
 '16': {'predicted': 'เป็นเกมส์ไทยของบริษัท MAD Virtual Reality Studio เป็นเกมส์แนวสยองขวัญ-ผจญภัย ผลิตออกมาในวันที่ 24 พฤศจิกายน พ.ศ. 2559 ให้ผู้เล่นสามารถทำภารกิจไขคดีปริศนาต่างๆและค้นหาอารยาที่หายตัวไปในโรงพยาบาล และผู้เล่นสามารถเล่นตัวละครทั้ง 3 คน',
  'question': 'อารยา เป็นเกมส์สัญชาติไทยที่ผลิตโดยบริษัทใด',
  'truth': 'MAD Virtual Reality Studio'},
 '18': {'predicted': '(ประตูใหญ่ทางทิศตะวันออก) คือประตูโบราณที่ตั้งอยู่ในโซล ประเทศเกาหลีใต้ มีชื่ออย่างเป็นทางการว่า ฮยุงอินจีมุน (흥인지문, Heunginjimun ประตูแห่งความเมตตากรุณา)',
  'question': 'ประตูโบราณดงแดมุนที่ตั้งอยู่ในกรุงโซล ประเทศเกาหลีใต้มีชื่ออย่างเป็นทางการว่าอะไร',
  'truth': 'ฮยุงอินจีมุน'},
 '19': {'p

In [49]:
eval_texts['incorrect_text']

{'10': {'predicted': 'ค.ศ. 1888',
  'question': 'ซิมโฟนีหมายเลข 2 หรือที่รู้จักกันในชื่อการฟื้นคืนชีพ เป็นผลงานประพันธ์ของใคร',
  'truth': 'กุสตาฟ มาห์เลอร์'},
 '11': {'predicted': 'War of Independence) สงครามนี้เริ่มต้นเมื่อวันที่ 16',
  'question': 'สงครามประกาศเอกราชเม็กซิโกเริ่มต้นเมื่อปี พ.ศ. อะไร',
  'truth': '2353'},
 '12': {'predicted': '大',
  'question': 'ตำแหน่งไดโจไดจิงหรืออัครมหาเสนาบดีได้ถูกยกเลิกและเปลี่ยนเป็นตำแหน่งนายกรัฐมนตรีในรัชสมัยของสมเด็จพระจักรพรรดิองค์ใด',
  'truth': 'เมจิ'},
 '14': {'predicted': 'จับอับ',
  'question': 'ตามที่มาของคำศัพท์ "จันอับ" เป็นชื่อเรียกของอะไร',
  'truth': 'กล่องใส่ขนม'},
 '17': {'predicted': 'ร้อยตำรวจตรี สุดเขต ประภากมล (ชื่อเล่น: เต่า; เกิด: 8',
  'question': 'สุดเขต ประภากมล เป็นนักกีฬาประเภทใด',
  'truth': 'แบดมินตัน'},
 '2': {'predicted': 'สาธารณรัฐไซปรัส',
  'question': 'ไซปรัสได้รับเอกราชจากการเป็นอาณานิคมของสหราชอาณาจักร ในปีใด',
  'truth': 'ปี 1960'},
 '20': {'predicted': 'Sound)',
  'question': 'กรันจ์เป็นแนวเพลงย่อยของแนวเพล

---

## 3.3 Evaluating Model

In [50]:
# A partial code from datasets.load_metric('squad')
# these functions are heavily influenced by the HF squad_metrics.py script
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

In [51]:
with open('test.json', 'r') as file:
  test = json.load(file, encoding='utf-8')

In [52]:
test[2]

{'context': 'โดราเอมอน โดราเอมอน หรือ โดเรมอน () () เป็น การ์ตูนญี่ปุ่น แต่งโดย ฟุจิโกะ ฟุจิโอะ เรื่องราวของหุ่นยนต์แมวชื่อโดราเอมอน โดยฟุจิโกะ ฟุจิโอะ ได้กล่าวว่าโดราเอมอนเกิดวันที่ 3 กันยายน มาจากอนาคตเพื่อกลับมาช่วยเหลือ โนบิตะ เด็กประถมจอมขี้เกียจด้วย ของวิเศษ จากอนาคต โดราเอมอนเริ่มตีพิมพ์ครั้งแรกในญี่ปุ่นเมื่อ เดือนมกราคม พ.ศ. 2513 โดย สำนักพิมพ์โชงะกุกัง ใน ประเทศไทย โดราเอมอนฉบับ หนังสือการ์ตูน มีการตีพิมพ์โดยหลายสำนักพิมพ์ในช่วงก่อนที่จะมีลิขสิทธิ์การ์ตูน แต่ปัจจุบัน สำนักพิมพ์ เนชั่น เอ็ดดูเทนเมนท์ เป็นผู้ได้รับลิขสิทธิ์ในการจัดพิมพ์แต่เพียงผู้เดียว ส่วนฉบับอะนิเมะ ออกอากาศครั้งแรก วันที่ 5 กันยายน พ.ศ. 2525 ทางโมเดิร์นไนน์ทีวี ในปัจจุบัน และวางจำหน่ายในรูปแบบวีซีดี-ดีวีดี ลิขสิทธิ์โดยบริษัท โรส วิดีโอ\xa0โครงเรื่อง โครงเรื่อง. เนื้อเรื่องส่วนมากจะเกี่ยวกับปัญหาของ โนบิตะ เด็กชายชั้น ป.5 ที่มักถูกเพื่อนๆ แกล้ง (แต่บ่อยครั้งก็เป็นฝ่ายหาเรื่องใส่ตัวเอง) ไม่ค่อยชอบทำการบ้าน, ไม่ชอบอ่านหนังสือและไปโรงเรียนสายบ่อยๆ โดยมีเพื่อนที่เป็นตัวละครสำคัญในเรื่องคือ โดราเอมอน (โนบิตะทำอะไรไ

In [53]:
text = test[2]['qas'][0]['answers'][0]['text']
text

'ฟุจิโกะ ฟุจิโอะ'

In [54]:
compute_exact_match(text, text), compute_exact_match(text, text[:-1])

(1, 0)

In [55]:
compute_f1(text, text), compute_f1(text, text[:-1])

(1.0, 0.5)

In [56]:
truth = [{'id': q['id'], 'answer' : q['answers'][0]['text']} for t in test for q in t['qas']]
truth[:10]

[{'answer': 'กรุงเทพมหานคร', 'id': '1'},
 {'answer': 'สะค้าน', 'id': '2'},
 {'answer': 'ฟุจิโกะ ฟุจิโอะ', 'id': '3'},
 {'answer': 'ท่าอากาศยานนานาชาติโหน่ยบ่าย', 'id': '4'},
 {'answer': 'บริษัท ไทยเบฟเวอร์เรจ จำกัด (มหาชน)', 'id': '5'},
 {'answer': 'ซวน กวิ่ญ', 'id': '6'},
 {'answer': 'สแตน ลี', 'id': '7'},
 {'answer': 'หอสมุดรัฐสภาอเมริกัน', 'id': '8'},
 {'answer': 'พระเจ้าบรมวงศ์เธอ พระองค์เจ้าศุขสวัสดี กรมหลวงอดิศรอุดมเดช',
  'id': '9'},
 {'answer': 'ซอยสีลม 5', 'id': '10'}]

In [57]:
test[0]

{'context': 'ถนนหทัยราษฎร์ ถนนหทัยราษฏร์ เป็นถนนที่เชื่อมระหว่างจังหวัดปทุมธานีกับกรุงเทพมหานคร โดยมีระยะทางประมาณ 15กม. โดยมีจุดเริ่มต้นที่สะพานสายไหมหทัยราษฎร์ หลังจากนั้นจะมุ่งหน้าไปทางทิศตะวันออก ข้ามถนนกาญจนาภิเษก แล้วเลี้ยวไปยังทิศตะวันออกเฉียงใต้โดยจะเข้าเขตคลองสามวาในพื้นที่กรุงเทพมหานครบริเวณซอยเลียบคลองสี่ ผ่านถนนราษฏร์นิมิตร คลองลำแบน ถนนไทยรามัญ คลองสี่ตะวันตก ซอยวัดแป้นทอง คลองสามตะวันตก คลองสองตะวันตก ผ่านแยกถนนเลียบคลองสอง ถนนหทัยมิตร แล้วถนนจะเข้าสู่พื้นที่เขตมีนบุรีผ่านย่านชุมชน ก่อนที่จะไปสิ้นสุดที่ถนนสุวินทวงศ์ลักษณะถนนลักษณะถนน. - ถนนสี่ช่องจราจรไปกลับแบบไม่มีเกาะกลาง ช่วงแรกตั้งแต่สามแยกสายไหม-หทัยราษฏร์จนถึงซอยหทัยราษฏร์ 37 และช่วงที่สอง ตั้งแต่ซอยหทัยราษฏร์ 27 จนถึง ถนนสุวินทวงศ์ - ถนนสี่ช่องจราจรไปกลับแบบมีเกาะกลาง ตั้งแต่ซอยหทัยราษฏร์ 37 จนถึงซอยหทัยราษฏร์ 27สถานที่สำคัญสถานที่สำคัญ. - โรงเรียนนวมินทราชินูทิศ เบญจมราชาลัย - มหาวิทยาลัยเวสเทิร์น วิทยาเขตวัชรพล\n',
 'qas': [{'answers': [{'answer_start': 69, 'text': 'กรุงเทพมหานคร'}],
   'id': '1',
   'is_impossib

In [58]:
import copy
temp = copy.deepcopy(test)

In [59]:
#predict model
predictions, probabilities = bert.predict(temp)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
add example index and unique id: 100%|██████████| 219/219 [00:00<00:00, 438031.75it/s]


Running Prediction:   0%|          | 0/39 [00:00<?, ?it/s]

In [60]:
pred = [{'id' : p['id'], 'answer' : p['answer'][0]} for p in predictions]
pred[:10]

[{'answer': '', 'id': '1'},
 {'answer': 'เอาะหลาม', 'id': '2'},
 {'answer': '', 'id': '3'},
 {'answer': 'ท่าอากาศยานนานาชาติโหน่ยบ่าย ท่าอากาศยานนานาชาติโหน่ยบ่าย () เป็นท่าอากาศยานที่ใหญ่ที่สุดในเวียดนามตอนเหนือ ให้บริการสำหรับบริเวณเมืองหลวงฮานอย ตั้งอยู่ห่างจากใจกลางเมือง 45 กิโลเมตร การเดินทางจากเมืองโดยรถแท็กซี่ใช้เวลา 30-45',
  'id': '4'},
 {'answer': '', 'id': '5'},
 {'answer': 'ซวน', 'id': '6'},
 {'answer': '', 'id': '7'},
 {'answer': 'มีหน้าที่รับจดทะเบียนลิขสิทธิ์', 'id': '8'},
 {'answer': '', 'id': '9'},
 {'answer': 'เป็นถนนซอยอยู่บนถนนสีลม', 'id': '10'}]

In [61]:
em = 0
f1 = []
for p, t in zip(pred, truth):
    # print(p, t)
    em += compute_exact_match(p['answer'], t['answer'])
    f1.append(compute_f1(p['answer'], t['answer']))
print(f'Exact match: {em/len(pred)}, F1-score: {np.mean(f1)}')

Exact match: 0.0365296803652968, F1-score: 0.06693121693121692


In [62]:
!zip -r 'best_model.zip' 'outputs/best_model' 
# !cp 'best_model.zip' {SAVE_DIR}

  adding: outputs/best_model/ (stored 0%)
  adding: outputs/best_model/training_args.bin (deflated 49%)
  adding: outputs/best_model/scheduler.pt (deflated 49%)
  adding: outputs/best_model/special_tokens_map.json (deflated 38%)
  adding: outputs/best_model/optimizer.pt (deflated 23%)
  adding: outputs/best_model/pytorch_model.bin (deflated 7%)
  adding: outputs/best_model/config.json (deflated 48%)
  adding: outputs/best_model/model_args.json (deflated 61%)
  adding: outputs/best_model/eval_results.txt (deflated 13%)
  adding: outputs/best_model/tokenizer_config.json (deflated 39%)
  adding: outputs/best_model/vocab.txt (deflated 72%)


In [63]:
#How to load model
loaded_model = QuestionAnsweringModel(
    'bert', 
    'outputs/best_model',
    use_cuda=torch.cuda.is_available()
    )

# answers, probabilities = loaded_model.predict(test)